# IE507 Lab 09
### Nonlinear Optimization with Pyomo
### October 06, 2021
(Narayan Rangaraj, Abijith P Y, Abhishek Chaudury)

---

One of the Pyomo's unique features is its ability to model nonlinear optimization problems. Several other Python packages are available for linear models (PuLP, CPLEX, Gurobi). However, Pyomo can be used for modeling general purpose nonlinear problems in Python also. 

Today we will try to model and solve some nonlinear problems. We first need to install a solver for nonlinear optimization. [Ipopt](http://projects.coin-or.org/Ipopt) is an open-source solver for constrained optimization problems. It is pronounced as "I--P--Opt". Like most other nonlinear solvers for continuous problems, it finds only locally minimum points. The solution it finds may not be globally optimal.

Ipopt has been compiled and made available online so that we may download and run it within this notebook. Ipopt also needs some fortran libraries which need to be installed. Let us install them.

In [1]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libgfortran5
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 589 kB of archives.
After this operation, 2,660 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libgfortran5 amd64 8.4.0-1ubuntu1~18.04 [589 kB]
Fetched 589 kB in 1s (809 kB/s)
Selecting previously unselected package libgfortran5:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libgfortran5_8.4.0-1ubuntu1~18.04_amd64.deb ...
Unpacking libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Setting up libgfortran5:amd64 (8.4.0-1ubuntu1~18.04) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



The above code also fetches an optimization problem in an '.nl' format. 'nl' format is suitable for solvers, not for humans. Lets call ipopt directly (without using any Python) to solve it.

In [2]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ


If you are able to see Ipopt output, we can install pyomo as usual

In [3]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 


In [4]:
import numpy as np

In [5]:
from pyomo.environ import *

Let us try to solve a nonlinear optimization problem without any constraints
$\min \ 100(x_2 - x_1^2)^2 + (1-x_1)^2$. Pyomo uses two asterisks (**) to denote 'raise to power'. Other mathematical operators like sqrt (for square root), log, sin, cos, multiplication and division etc are available.

In [6]:
model = ConcreteModel()
model.x1 = Var()
model.x2 = Var()

model.obj = Objective(expr = 100.0*(model.x2-model.x1**2)**2 + (1-model.x1)**2, sense=minimize)
model.pprint() 


2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 100.0*(x2 - x1**2)**2 + (1 - x1)**2

3 Declarations: x1 x2 obj



Next load the solver and pass the model to it, solve and get the results, just like you did for linear optimization.

In [7]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value)  


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.027719974517822266
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  1.3288608467480825e-28

Decision Variables
x1 =  0.9999999999999899
x2 =  0.9999999999999792


In [8]:
model.constraints = ConstraintList()

In [9]:
model.constraints.add(expr = model.x1 >=5)
model.constraints.add(expr = model.x1 <=10)
model.constraints.add(expr = model.x2 >=5)
model.constraints.add(expr = model.x2 <=10)

In [10]:
result = opt_ipopt.solve(model)
print(result)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())


print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02461528778076172
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  22515.998199605063

Decision Variables
x1 =  4.999999950000084
x2 =  10.000000099999165


In [11]:
model.constraints.add(expr = model.x1**2 - 14*model.x1 + model.x2**2 - 12*model.x2 <= -83)

In [12]:
result = opt_ipopt.solve(model)
print(result)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())
u = model.obj()

print("\nConstraints:")
print(model.constraints.display())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.026129961013793945
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  63212.32973634936

Constraints:
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :   5.0 :  5.591404989462342 :  None
      2 :  None :  5.591404989462342 :  10.0
      3 :   5.0 :  6.125940169329545 :  None
      4 :  None :  6.125940169329545 :  10.0
      5 :  None : -82.99999917003765 : -83.0
None

Decision Variables
x1 =  5.591404989462342
x2 =  6.125940169329545


In the above result we can see that only constraint 5 is active ,all ohters are inactive.

In [13]:
for i in range(1,5):
  model.constraints[i].deactivate()

In [14]:
result = opt_ipopt.solve(model)
print(result)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())
v = model.obj()

print("\nConstraints:")
print(model.constraints.display())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.0194854736328125
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  63212.329737006454

Constraints:
constraints : Size=5
    Key : Lower : Body               : Upper
      5 :  None : -82.99999917007058 : -83.0
None

Decision Variables
x1 =  5.591404989472643
x2 =  6.125940169314059


In [15]:
print("change in optimal solution : u - v = ", u-v)

change in optimal solution : u - v =  -6.570917321369052e-07


We can see amlost negligible change in optimal solution.